# Токсичные комметарии
Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.
_______________________________________

##### [1. Подготовка данных](#Part_1)

##### [2. Обучение моделей](#Part_2)
* [2.1 Логистическая регрессия](#Part_2.1)
* [2.2 Случайный лес ](#Part_2.2)
* [2.3 Градиентый бустинг](#Part_2.3)

##### [3. Выводы](#Part_3)

<a id='Part_1'></a>
# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
import nltk
import re
from tqdm import tqdm, notebook

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df['toxic'].mean() #процент токсичных комментариев

0.10167887648758234

Проведем очистку и лемматизацию текста.

In [6]:
def clear_text(text):
    cl_text = re.sub(r"[^a-zA-Z' ]", ' ', text)
    cl_text = cl_text.split()
    return ' '.join(cl_text)

In [7]:
def lemmatize(text):
    text = clear_text(text)
    word_list = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(text) for text in word_list])

In [8]:
notebook.tqdm.pandas()

df['lemma'] = df['text'].progress_apply(lambda x: clear_text(lemmatize(x)))

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [9]:
range(df.shape[0])

range(0, 159571)

<a id='Part_2'></a>
# 2. Обучение моделей

In [10]:
train, test = train_test_split(df, shuffle=True, test_size=0.2, random_state=42)

In [11]:
train.shape, test.shape

((127656, 3), (31915, 3))

Векторизируем лемматизированный текст.

In [12]:
corpus = train['lemma']
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_features_train = TfidfVectorizer(stop_words=stopwords)

features_train = count_features_train.fit_transform(corpus) 

print("Размер матрицы:", features_train.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размер матрицы: (127656, 143930)


In [13]:
features_test = count_features_train.transform(test['lemma'])

target_train = train['toxic']
target_test = test['toxic']

<a id='Part_2.1'></a>
## 2.1 Логистическая регрессия

In [14]:
logic = LogisticRegression().fit(features_train,target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [15]:
# выберем порог определения класса по трейну и выставим его для предсказания теста
def prob(model, lr, hr, step, feature, target):
    probabilities_one_valid = model.predict_proba(feature)[:, 1]

    for threshold in np.arange(lr, hr, step):
        predicted_valid = probabilities_one_valid > threshold 
        f1 = f1_score(target, pd.Series(predicted_valid))
    
        print("Порог = {:.2f} | f1 = {:.3f}".format(threshold, f1))

In [16]:
prob(logic, 0.2, 0.3, 0.01, features_train, target_train)

Порог = 0.20 | f1 = 0.831
Порог = 0.21 | f1 = 0.833
Порог = 0.22 | f1 = 0.835
Порог = 0.23 | f1 = 0.836
Порог = 0.24 | f1 = 0.835
Порог = 0.25 | f1 = 0.837
Порог = 0.26 | f1 = 0.836
Порог = 0.27 | f1 = 0.834
Порог = 0.28 | f1 = 0.832
Порог = 0.29 | f1 = 0.829


In [17]:
predict = pd.Series((logic.predict_proba(features_test)[:,1]>0.25).astype(int))

In [18]:
f1_score(target_test, predict)

0.7828833096478762

<a id='Part_2.2'></a>
## 2.2 Случайный лес

In [19]:
forest = RandomForestClassifier(random_state=42, n_estimators = 100).fit(features_train, target_train)

In [20]:
prob(forest, 0.25, 0.35, 0.01, features_test, target_test)

Порог = 0.25 | f1 = 0.777
Порог = 0.26 | f1 = 0.780
Порог = 0.27 | f1 = 0.782
Порог = 0.28 | f1 = 0.785
Порог = 0.29 | f1 = 0.784
Порог = 0.30 | f1 = 0.781
Порог = 0.31 | f1 = 0.782
Порог = 0.32 | f1 = 0.779
Порог = 0.33 | f1 = 0.777
Порог = 0.34 | f1 = 0.774


In [21]:
predict = pd.Series((forest.predict_proba(features_test)[:,1]>0.28).astype(int))
f1_score(target_test, predict)

0.7845181674565561

<a id='Part_2.3'></a>
## 2.3 Градиентный бустинг

In [22]:
# model
gbm = lgb.LGBMClassifier(learning_rate=0.38,
                         n_estimators=300,
                        random_state=42
                       )
# fit
gbm.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='logloss',
        early_stopping_rounds=5,
        verbose = 20)

Training until validation scores don't improve for 5 rounds
[20]	training's binary_logloss: 0.11995
[40]	training's binary_logloss: 0.100453
[60]	training's binary_logloss: 0.0883308
[80]	training's binary_logloss: 0.0794595
[100]	training's binary_logloss: 0.0722971
[120]	training's binary_logloss: 0.0663133
[140]	training's binary_logloss: 0.0612156
[160]	training's binary_logloss: 0.0568776
[180]	training's binary_logloss: 0.0529086
[200]	training's binary_logloss: 0.0494532
[220]	training's binary_logloss: 0.0462622
[240]	training's binary_logloss: 0.0434855
[260]	training's binary_logloss: 0.0409287
[280]	training's binary_logloss: 0.0385425
[300]	training's binary_logloss: 0.0364071
Did not meet early stopping. Best iteration is:
[300]	training's binary_logloss: 0.0364071


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.38, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [23]:
prob(gbm, 0.35, 0.45, 0.01, features_test, target_test)

Порог = 0.35 | f1 = 0.777
Порог = 0.36 | f1 = 0.777
Порог = 0.37 | f1 = 0.777
Порог = 0.38 | f1 = 0.778
Порог = 0.39 | f1 = 0.778
Порог = 0.40 | f1 = 0.777
Порог = 0.41 | f1 = 0.777
Порог = 0.42 | f1 = 0.776
Порог = 0.43 | f1 = 0.776
Порог = 0.44 | f1 = 0.776
Порог = 0.45 | f1 = 0.775


In [24]:
predict = pd.Series((gbm.predict_proba(features_test)[:,1]>0.38).astype(int))
f1_score(target_test, predict)

0.7777958123681221

<a id='Part_3'></a>
# 3. Выводы

#### Для построения модели классификация твиттов по токсичности, мы проделали следующие шаги:
* текст очищен от посторонних символов, оставлен только английский алфавит
* текст твиттов лемматизирован
* текст переведен в векторную форму с помощью TfidfVectorizer
* обучены три модели классификации
* на всех трех моделях удалось достичь метрику f1 больше 0.75
* Самой быстрой моделью стала логистическая регрессия, а самой точной случайный лес